# Building Recommendation Engines with PySpark

This course will show you how to build recommendation engines using `Alternating Least Squares` in PySpark. Using the popular MovieLens dataset and the Million Songs dataset, this course will take you step by step through the intuition of the Alternating Least Squares algorithm as well as the code to train, test and implement ALS models on various types of customer data.

## Table of Contents

- [Introduction](#intro)
- 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

path = "data/dc36/"

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")
print(sc)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('First App').getOrCreate()

In [ ]:
# Return spark version
print(spark.version)

# Return python version
import sys
print(sys.version_info)

---
<a id='intro'></a>

## Why learn how to build recommendation engines?

<img src="images/spark6_001.png" alt="" style="width: 800px;"/>

<img src="images/spark6_002.png" alt="" style="width: 800px;"/>

<img src="images/spark6_003.png" alt="" style="width: 800px;"/>

## 

In [ ]:
# Terminate the cluster
spark.stop()

In [ ]:
<img src="images/spark6_004.png" alt="" style="width: 800px;"/>

In [ ]:
---
<a id='intro'></a>